In [1]:
import os
import pandas as pd

In [2]:
df = pd.read_csv('data\c4_cognitive_score_mri.csv')
df.head()

,subject_id,VISCODE,VISCODE2,EXAMDATE,DIAGNOSIS,image_id,mri_visit,mri_date,mri_description,image_type,mri_type,days_between_exams_and_mri_scan
0,002_S_0413,init,m132,2017-06-15,1.0,863056,init,2017-06-21,Accelerated Sagittal MPRAGE,Original,3D,6
1,002_S_1155,init,m126,2017-04-20,2.0,843510,init,2017-04-24,Accelerated Sagittal MPRAGE,Original,3D,4
2,002_S_1261,init,m120,2017-03-14,2.0,831065,init,2017-03-15,Accelerated Sagittal MPRAGE,Original,3D,1
3,002_S_1280,init,m120,2017-03-02,1.0,829296,init,2017-03-13,Accelerated Sagittal MPRAGE,Original,3D,11
4,002_S_4225,init,m84,2018-11-08,3.0,1075136,init,2018-11-12,Accelerated Sagittal MPRAGE,Original,3D,4


In [3]:
df = df[~(df['VISCODE'] == 'sc')]

In [4]:
# df = df[df['days_between_exams_and_mri_scan'] <= 30]

In [5]:
df.image_type.unique()

array(['Original'], dtype=object)

In [6]:
df.mri_type.unique()

array(['3D'], dtype=object)

In [7]:
df.mri_visit.unique()

array(['init', 'sc', 'y4', 'y2', 'y1', 'y3', 'y5', 'v31'], dtype=object)

In [8]:
# Xóa các cột thừa gây nhiễu
df = df.drop(columns=['VISCODE', 'VISCODE2', 'EXAMDATE', 'days_between_exams_and_mri_scan', 'image_type', 'mri_type', 'mri_visit'])

In [9]:
df = df.drop_duplicates('image_id')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 205 entries, 0 to 291
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   subject_id       205 non-null    object 
 1   DIAGNOSIS        205 non-null    float64
 2   image_id         205 non-null    int64  
 3   mri_date         205 non-null    object 
 4   mri_description  205 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 9.6+ KB


In [11]:
df.count()

subject_id         205
DIAGNOSIS          205
image_id           205
mri_date           205
mri_description    205
dtype: int64

In [12]:
df.columns

Index(['subject_id', 'DIAGNOSIS', 'image_id', 'mri_date', 'mri_description'], dtype='object')

In [13]:
df.mri_description.unique()

array(['Accelerated Sagittal MPRAGE', 'Accelerated Sagittal MPRAGE_ND',
       'Sagittal 3D Accelerated MPRAGE', 'MPRAGE SAG', 'MPRAGE',
       'MPRAGE GRAPPA2', 'Accelerated Sagittal MPRAGE REPEAT'],
      dtype=object)

In [14]:
df.mri_description.value_counts()

mri_description
Accelerated Sagittal MPRAGE           191
Sagittal 3D Accelerated MPRAGE          6
Accelerated Sagittal MPRAGE_ND          4
MPRAGE SAG                              1
MPRAGE                                  1
MPRAGE GRAPPA2                          1
Accelerated Sagittal MPRAGE REPEAT      1
Name: count, dtype: int64

In [15]:
# danh sách các mẫu dữ liệu chụp lại
df1 = df[df['mri_description'].isin(['Accelerated Sagittal MPRAGE repeat', 'Accelerated Sagittal MPRAGE REPEAT'])]
df1

,subject_id,DIAGNOSIS,image_id,mri_date,mri_description
250,137_S_4631,3.0,1116492,2019-01-15,Accelerated Sagittal MPRAGE REPEAT


In [16]:
# lấy những ảnh bị lỗi phải chụp lại
df2 = df[(df['subject_id'].isin(df1['subject_id']))]
df2 = df2[~df2['mri_description'].isin(['Accelerated Sagittal MPRAGE repeat', 'Accelerated Sagittal MPRAGE REPEAT'])]
df2

,subject_id,DIAGNOSIS,image_id,mri_date,mri_description
249,137_S_4631,3.0,1116491,2019-01-15,Accelerated Sagittal MPRAGE


In [17]:
# lấy những ảnh loại axial dti_fa không bị lỗi
df3 = df[df['mri_description'].isin(['Accelerated Sagittal MPRAGE', 'Sagittal 3D Accelerated MPRAGE',
                                     'Accelerated Sagittal MPRAGE REPEAT', 'Accelerated Sagittal MPRAGE repeat' ])]
df3 = df3[~df3['image_id'].isin(df2['image_id'])]

In [18]:
df3.count()

subject_id         197
DIAGNOSIS          197
image_id           197
mri_date           197
mri_description    197
dtype: int64

In [19]:
# Tạo 1 df trống gồm các cột: subject_id, mri_description, mri_time, image_id
mri_df = pd.DataFrame(columns=['subject_id', 'mri_description', 'mri_time', 'image_id'])

In [20]:
data_dir = 'data/MRI' 

for f1 in os.listdir(data_dir):
    if os.path.isdir(os.path.join(data_dir, f1)):
        # print(f1)
        for f2 in os.listdir(os.path.join(data_dir, f1)):
            if os.path.isdir(os.path.join(data_dir, f1, f2)):
                # print(f2)
                for f3 in os.listdir(os.path.join(data_dir, f1, f2)):
                    if os.path.isdir(os.path.join(data_dir, f1, f2, f3)):
                        for f4 in os.listdir(os.path.join(data_dir, f1, f2, f3)):
                            if os.path.isdir(os.path.join(data_dir, f1, f2, f3, f4)):
                                mri_df = pd.concat([mri_df, pd.DataFrame({'subject_id': [f1], 'mri_description': [f2], 'mri_time': [f3], 'image_id': [f4]})], ignore_index=True)

In [21]:
mri_df.head()

,subject_id,mri_description,mri_time,image_id
0,002_S_0413,Accelerated_Sagittal_MPRAGE,2017-06-21_13_23_38.0,I863056
1,002_S_1155,Accelerated_Sagittal_MPRAGE,2017-04-24_13_21_32.0,I843510
2,002_S_1261,Accelerated_Sagittal_MPRAGE,2017-03-15_11_23_54.0,I831065
3,002_S_1280,Accelerated_Sagittal_MPRAGE,2017-03-13_13_38_31.0,I829296
4,002_S_4225,Accelerated_Sagittal_MPRAGE,2018-11-12_10_09_12.0,I1075136


In [22]:
mri_df.count()

subject_id         243
mri_description    243
mri_time           243
image_id           243
dtype: int64

In [23]:
mri_df.mri_description.value_counts()

mri_description
Accelerated_Sagittal_MPRAGE           226
Sagittal_3D_Accelerated_MPRAGE          8
Accelerated_Sagittal_MPRAGE_ND          4
MPRAGE                                  1
MPRAGE_GRAPPA2                          1
MPRAGE_SAG                              1
Accelerated_Sagittal_MPRAGE_REPEAT      1
Accelerated_Sagittal_MPRAGE_repeat      1
Name: count, dtype: int64

In [24]:
df3['image_id'] = 'I' + df3['image_id'].astype(str)

In [25]:
# chỉ lấy các mẫu dữ liệu thực sự tồn tại:

df4 = pd.merge(mri_df, df3, on=['image_id'], how='inner')
df4.head()

,subject_id_x,mri_description_x,mri_time,image_id,subject_id_y,DIAGNOSIS,mri_date,mri_description_y
0,002_S_0413,Accelerated_Sagittal_MPRAGE,2017-06-21_13_23_38.0,I863056,002_S_0413,1.0,2017-06-21,Accelerated Sagittal MPRAGE
1,002_S_1155,Accelerated_Sagittal_MPRAGE,2017-04-24_13_21_32.0,I843510,002_S_1155,2.0,2017-04-24,Accelerated Sagittal MPRAGE
2,002_S_1261,Accelerated_Sagittal_MPRAGE,2017-03-15_11_23_54.0,I831065,002_S_1261,2.0,2017-03-15,Accelerated Sagittal MPRAGE
3,002_S_1280,Accelerated_Sagittal_MPRAGE,2017-03-13_13_38_31.0,I829296,002_S_1280,1.0,2017-03-13,Accelerated Sagittal MPRAGE
4,002_S_4225,Accelerated_Sagittal_MPRAGE,2018-11-12_10_09_12.0,I1075136,002_S_4225,3.0,2018-11-12,Accelerated Sagittal MPRAGE


In [26]:
df4.count()

subject_id_x         197
mri_description_x    197
mri_time             197
image_id             197
subject_id_y         197
DIAGNOSIS            197
mri_date             197
mri_description_y    197
dtype: int64

In [27]:
# đổi tên cột:
df4 = df4.rename(columns={'subject_id_x': 'subject_id'})

In [28]:
df4['mri_link'] =  "data" + "\\" + "MRI" + "\\" + df4['subject_id'] + "\\" + df4['mri_description_x'] + "\\" + df4['mri_time'] + "\\" + df4['image_id']

In [29]:
# xóa cột thừa:
df4 = df4.drop(columns=['subject_id_y', 'mri_description_y', 'mri_description_x', 'image_id', 'mri_time'])

In [30]:
df4.head()

,subject_id,DIAGNOSIS,mri_date,mri_link
0,002_S_0413,1.0,2017-06-21,data\MRI\002_S_0413\Accelerated_Sagittal_MPRAG...
1,002_S_1155,2.0,2017-04-24,data\MRI\002_S_1155\Accelerated_Sagittal_MPRAG...
2,002_S_1261,2.0,2017-03-15,data\MRI\002_S_1261\Accelerated_Sagittal_MPRAG...
3,002_S_1280,1.0,2017-03-13,data\MRI\002_S_1280\Accelerated_Sagittal_MPRAG...
4,002_S_4225,3.0,2018-11-12,data\MRI\002_S_4225\Accelerated_Sagittal_MPRAG...


In [31]:
df4.DIAGNOSIS.value_counts()

DIAGNOSIS
1.0    124
2.0     52
3.0     21
Name: count, dtype: int64

In [32]:
# lưu lại df1 vào file csv
df4.to_csv('data/mri.csv', index=False)